# CS5346 Mini-Project 2 - Data Processing
---

In [8]:
import numpy as np
import datetime as dt
import pandas as pd
import json

In [5]:
with open('Mini-project-2-data.json', "r", encoding='utf-8') as json_data:
    raw_data = json.load(json_data)

In [6]:
len(raw_data)

200000

## Dataset 1
__arXiv__
- Author ID
- Autho Name
- Year
- Venue
- Journal Name
- In Citations

In [78]:
temp = pd.DataFrame(raw_data)
arXiv = temp.loc[temp['venue'] == "ArXiv"]
arXiv = arXiv[["authors", "id", "year", "inCitations"]]
arXiv["inCitations"] = arXiv["inCitations"].apply(lambda x: len(x))
arXiv["year"] = arXiv["year"].apply(lambda x: int(x))

In [79]:
temp = arXiv.set_index(['id', 'year', 'inCitations'])
unnested_lst = []
for col in temp.columns:
    unnested_lst.append(temp[col].apply(pd.Series).stack())
result = pd.concat(unnested_lst, axis=1, keys=temp.columns)
result = result.reset_index()
result["author_id"] = result["authors"].apply(lambda x: x["ids"][0] if len(x["ids"]) > 0 else "")
result["author_name"] = result["authors"].apply(lambda x: x["name"])
result = result.drop(['level_3', 'authors'], axis = 1)

In [80]:
result.to_csv('obj1.csv', sep=',', encoding='utf-8', index=False)

## Dataset 2
__arXiv__
- Year
- Venue
- In Citations

In [81]:
temp = pd.DataFrame(raw_data)
arXiv = temp.loc[temp['venue'] == "ArXiv"]
arXiv = arXiv[["id", "year", "inCitations"]]

In [82]:
arXiv.to_csv('obj2.csv', sep=',', encoding='utf-8', index=False)

## Dataset 3
__ICSE__
- Authors
- Venue
- In Citations
- Out Citations

In [89]:
temp = pd.DataFrame(raw_data)

ICSE = temp.loc[temp['venue'].str.startswith("ICSE")]
ICSE = ICSE[["id", "authors", "year", "inCitations", "outCitations"]]
ICSE.count()

id              95
authors         95
year            95
inCitations     95
outCitations    95
dtype: int64

In [90]:
ICSE.to_csv('obj3.csv', sep=',', encoding='utf-8', index=False)